In [ ]:
import numpy as np
import pandas as pd
import json
import swifter

data_dir = 'D:/beibei/database/beibei_rebate/'

In [ ]:
rebate_order_info = pd.read_csv(data_dir + 'rebate_order_info_6_9__80c8f135_7fda_461d_989a_91c255c711c5')
rebate_order_extra = pd.read_csv(data_dir + 'rebate_order_extra_6_9__5f4605c1_9857_4ec3_8806_be806ab62c4f')

In [ ]:
# order_info = pd.concat([rebate_order_info, rebate_order_extra], axis=1)
order_info = rebate_order_info.merge(rebate_order_extra, left_on='id', right_on='oid', suffixes=('_info', '_extra'), how='left')

In [ ]:
def get_category(row):
    try:
        if row['item_info'] is None:
            return None
        item_info = json.loads(row['item_info'])
        category = item_info.get('itemCategoryName')
        if category is None:
            category = item_info.get('cid1Name')
        return category
    except Exception as e:
        pass
    return None

order_info['item_category'] = order_info.swifter.apply(lambda row: get_category(row), axis=1)


In [ ]:
count_df = order_info[['item_category', 'uid', 'trade_id']]
counted_df = count_df.groupby(by=['item_category'], as_index=False)['trade_id'].count()

In [ ]:
counted_df = counted_df.sort_values(by=['trade_id'], ascending=False)
counted_df.to_csv(data_dir + 'category-rank-2022.6-9.csv')

In [ ]:
valid_order = order_info.loc[(order_info['order_status'].isin([1,3,4,6])) & (order_info['o_tags'].str.contains('SELF'))]



In [ ]:
def save_tmp_order():
    tmp = valid_order.loc[(valid_order['uid'] == 105532235) & (valid_order['item_category'] == '童装/婴儿装/亲子装')]
    tmp.to_csv('tmp.csv')


In [ ]:

women_dress = valid_order.loc[valid_order['item_category'] == '女装/女士精品']
kid_dress = valid_order.loc[valid_order['item_category'] == '童装/婴儿装/亲子装']


for idx, category_order in enumerate([women_dress, kid_dress]):
    category_count_df = category_order[['item_category', 'uid', 'trade_id']]
    category_counted_df = category_count_df.groupby(by=['uid'], as_index=False)['trade_id'].count()
    category_counted_df = category_counted_df.sort_values(by=['trade_id'], ascending=False)
    
    filename = '%soutput/category-rank-%s-2022.6-9.csv' % (data_dir, idx)
    category_counted_df.to_csv(filename)


